# Classify whether infinity or days

In [42]:
df = pd.read_csv("dataset/protected_features.csv")

df

,page_id,page_title,view_count,protection_level,Unnamed: 4,page_length,number_page_watchers,number_page_watchers_recent_edits,number_of_redirects,page_views_past_30days,total_edits,recent_number_of_edits,number_distinct_authors,number_categories,protection_expiry,protection_expiry_date,protection_expiry_time,protection_duration_days,timestamp,protection_expiry_flag
0,18938265,"""Weird Al"" Yankovic",NaN,autoconfirmed,NaN,23405,93,24,0,59749.0,665.0,44.0,38.0,13.0,2021-08-18T01:18:23Z,2021-08-18,01:18:23Z,-2.0,2021-08-20,0
1,5352780,(A Ballad of) A Peaceful Man,NaN,autoconfirmed,NaN,9145,Fewer than 30 watchers,NaN,2,5920.0,422.0,12.0,6.0,10.0,2021-08-18T02:09:11Z,2021-08-18,02:09:11Z,-2.0,2021-08-20,0
2,57057212,(G)I-dle,NaN,autoconfirmed,NaN,93938,106,19,9,20005.0,1637.0,96.0,15.0,24.0,2021-08-18T02:15:06Z,2021-08-18,02:15:06Z,-2.0,2021-08-20,0
3,31455534,...In Spite of Harry's Toenail,NaN,autoconfirmed,NaN,159895,417,42,11,102365.0,12536.0,78.0,37.0,24.0,2021-08-18T03:09:46Z,2021-08-18,03:09:46Z,-2.0,2021-08-20,0
4,290328,.org,NaN,autoconfirmed,NaN,145575,91,13,7,9649.0,4807.0,164.0,40.0,4.0,2021-08-18T03:14:08Z,2021-08-18,03:14:08Z,-2.0,2021-08-20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12478,21314193,Mystic God,NaN,sysop,NaN,32,Fewer than 30 watchers,NaN,0,1.0,3.0,0.0,0.0,NaN,infinity,infinity,31-12-9999,NaN,2021-08-20,1
12479,28995383,Mythological creature list,NaN,sysop,NaN,42,Fewer than 30 watchers,NaN,0,24.0,8.0,0.0,0.0,NaN,infinity,infinity,31-12-9999,NaN,2021-08-20,1
12480,20596380,Mzoli's Meats,NaN,sysop,NaN,21,Fewer than 30 watchers,NaN,0,8.0,2.0,0.0,0.0,NaN,infinity,infinity,31-12-9999,NaN,2021-08-20,1
12481,28262947,Ménage à 3,NaN,sysop,NaN,30,Fewer than 30 watchers,NaN,0,285.0,2.0,0.0,0.0,NaN,infinity,infinity,31-12-9999,NaN,2021-08-20,1


In [27]:
df['protection_expiry_date']

0         2021-08-18
1         2021-08-18
2         2021-08-18
3         2021-08-18
4         2021-08-18
            ...     
12478     9999-12-31
12479     9999-12-31
12480     9999-12-31
12481     9999-12-31
12482     9999-12-31
Name: protection_expiry_date, Length: 12483, dtype: object

In [28]:
#df_test['Difference'] = (df_test['First_Date'] - df_test['Second Date']).dt.days


pd.to_datetime(df['timestamp']) - pd.to_datetime(df['protection_expiry_date'])

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 9999-12-31 00:00:00

In [35]:
temp = df.loc[1:2286]

In [37]:
difference = pd.to_datetime(temp['protection_expiry_date']) - pd.to_datetime(temp['timestamp']) 

In [39]:
difference

1        -2 days
2        -2 days
3        -2 days
4        -2 days
5        -2 days
          ...   
2282   1764 days
2283   1871 days
2284   2453 days
2285   2691 days
2286   2742 days
Length: 2286, dtype: timedelta64[ns]

In [41]:
difference.to_csv("dataset/difference-days.csv", index=False)

In [52]:
import lazypredict
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

#Read data file

import pandas as pd

filepath = "dataset/protected_features.csv"
df = pd.read_csv(filepath)
features = df

# Labels are the values we want to predict
labels = np.array(df['protection_expiry_flag'])

# 0 => protection date exists
# 1 => infinity

# Remove the labels from the features
list_dropped_columns = ['protection_level', 
                        'page_title', 
                        'protection_expiry',
                        'protection_expiry_flag',
                        'timestamp',
                        'protection_expiry_date', 
                        'protection_expiry_time',
                        'protection_expiry',
                        'protection_expiry.1'
                        ]

features = features.drop(list_dropped_columns, axis=1)
# features = features.drop('protection_expiry', axis=1)
# features = features.drop('page_id', axis = 1)

# Replace NaN
features = features.replace('Fewer than 30 watchers',np.NaN)
features = features.replace('There may or may not be a watching user visiting recent edits',np.NaN)

#Convert cols to Float
features['page_length'] = features['page_length'].astype(float)
features['total_edits'] = features['total_edits'].astype(float)
features['number_page_watchers'] = features['number_page_watchers'].astype(float)
features['number_page_watchers_recent_edits'] = features['number_page_watchers_recent_edits'].astype(float)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

features

array([[ 1.8938265e+07,            nan,  2.3405000e+04,  9.3000000e+01,
         2.4000000e+01,  0.0000000e+00,  5.9749000e+04,  6.6500000e+02,
         4.4000000e+01,  3.8000000e+01,  1.3000000e+01, -2.0000000e+00],
       [ 5.3527800e+06,            nan,  9.1450000e+03,            nan,
                   nan,  2.0000000e+00,  5.9200000e+03,  4.2200000e+02,
         1.2000000e+01,  6.0000000e+00,  1.0000000e+01, -2.0000000e+00],
       [ 5.7057212e+07,            nan,  9.3938000e+04,  1.0600000e+02,
         1.9000000e+01,  9.0000000e+00,  2.0005000e+04,  1.6370000e+03,
         9.6000000e+01,  1.5000000e+01,  2.4000000e+01, -2.0000000e+00],
       [ 3.1455534e+07,            nan,  1.5989500e+05,  4.1700000e+02,
         4.2000000e+01,  1.1000000e+01,  1.0236500e+05,  1.2536000e+04,
         7.8000000e+01,  3.7000000e+01,  2.4000000e+01, -2.0000000e+00],
       [ 2.9032800e+05,            nan,  1.4557500e+05,  9.1000000e+01,
         1.3000000e+01,  7.0000000e+00,  9.6490000e+03,  4.8

In [58]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size =0.20, random_state = 53)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels



from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:25<00:00,  1.12it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
XGBClassifier                      0.91               0.78     0.78      0.90   
LGBMClassifier                     0.91               0.78     0.78      0.90   
BaggingClassifier                  0.89               0.78     0.78      0.89   
RandomForestClassifier             0.90               0.77     0.77      0.89   
AdaBoostClassifier                 0.90               0.76     0.76      0.89   
DecisionTreeClassifier             0.84               0.74     0.74      0.84   
ExtraTreesClassifier               0.89               0.73     0.73      0.88   
KNeighborsClassifier               0.87               0.70     0.70      0.86   
LabelPropagation                   0.85               0.69     0.69      0.84   
LabelSpreading                     0.85               0.68     0.68      0.84   
NearestCentroid             

In [65]:
models['Accuracy']


Model
XGBClassifier                   0.91
LGBMClassifier                  0.91
BaggingClassifier               0.89
RandomForestClassifier          0.90
AdaBoostClassifier              0.90
DecisionTreeClassifier          0.84
ExtraTreesClassifier            0.89
KNeighborsClassifier            0.87
LabelPropagation                0.85
LabelSpreading                  0.85
NearestCentroid                 0.81
BernoulliNB                     0.80
SVC                             0.87
ExtraTreeClassifier             0.79
PassiveAggressiveClassifier     0.78
QuadraticDiscriminantAnalysis   0.84
Perceptron                      0.81
GaussianNB                      0.84
LogisticRegression              0.85
CalibratedClassifierCV          0.84
LinearSVC                       0.84
SGDClassifier                   0.84
LinearDiscriminantAnalysis      0.84
RidgeClassifierCV               0.83
RidgeClassifier                 0.83
DummyClassifier                 0.70
Name: Accuracy, dtype: float64

In [70]:
#X_train.isnan()

array_sum = np.sum(X_train)
array_has_nan = np.isnan(array_sum)
array_has_nan

False

In [72]:
# Random Forest Classifier implementation 

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

#Predict
y_pred = rf.predict(X_test)
predictions = [round(value) for value in y_pred]


#Evaluate
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.4f%%" % (accuracy * 100.0))

Accuracy: 90.2683%
